In [1]:
import sys
import aux_func
import pandas as pd
import numpy as  np
import imageio
import polars as pl
from tqdm import tqdm
from PyQt5.QtWidgets import QWidget, QHBoxLayout
import visvis as vv

In [2]:
DL_PATH = catalog.load("params:data_dwld")
DATA_URL = catalog.load("params:data_url_67p")

aux_func.download_file(DL_PATH, DATA_URL)

[02/09/24 00:29:45] INFO     Loading data from 'params:data_dwld' (MemoryDataset)...            ]8;id=880998;file:///home/rafael/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=487130;file:///home/rafael/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    INFO     Loading data from 'params:data_url_67p' (MemoryDataset)...         ]8;id=431941;file:///home/rafael/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=546232;file:///home/rafael/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

In [3]:
data_dir = catalog.load("params:data")
col_names = catalog.load("params:col_names")


df = aux_func.obj2df(data_dir, col_names)
df.head()

[02/09/24 00:29:46] INFO     Loading data from 'params:data' (MemoryDataset)...                 ]8;id=278629;file:///home/rafael/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=570738;file:///home/rafael/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    INFO     Loading data from 'params:col_names' (MemoryDataset)...            ]8;id=222274;file:///home/rafael/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=552408;file:///home/rafael/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

type,x1,x2,x3
str,f64,f64,f64
"""v""",0.570832,-1.000444,0.532569
"""v""",0.56436,-1.000224,0.52536
"""v""",0.557853,-0.997863,0.520762
"""v""",0.553592,-0.998414,0.512192
"""v""",0.550212,-0.992514,0.507304


In [4]:
print("Information about the shapoe model: ")
print(f"Rows and columns of the dataset {df.shape}")

vertices_faces = df['type'].value_counts()
print(f"# of faces: {vertices_faces[0]['count'][0]}")
print(f"# of vertices: {vertices_faces[1]['count'][0]}")

Information about the shapoe model: 
Rows and columns of the dataset (1791747, 4)
# of faces: 597251
# of vertices: 1194496


In [5]:
vertices = (
    df
    .filter(pl.col("type") == "v")
    .select(["x1", "x2", "x3"])
    .to_numpy()
    .tolist()
)

faces = (
    df
    .filter(pl.col("type") == "f")
    .select(["x1", "x2", "x3"])
    .to_pandas()
    .values
)


The indexes sould start at s0 in Python, substract 1 is needed.

In [6]:
faces = faces - 1
faces = faces.astype(int)
faces = faces.tolist()

Now is time to adjust the resolution of the window for the 3D video:

In [ ]:
app = vv.use()
app.Create()

main = aux_func.MainViewPanel()
main.resize(500, 400)

vv.mesh(vertices=vertices, faces=faces, verticesPerFace=3)

ax = vv.gca()
ax.bgcolor = 'black'

ax.axis.showGrid = False
ax.axis.visible = False
ax.camera = "3d"
ax.camera.fov = 60
ax.camera.azimuth = 120
ax.camera.elevation = 25

app.Run()

In [ ]:
app = vv.use()
app.Create()

main_w = MainWindow()
main_w.resize(500, 400)


shape_obj = vv.mesh(vertices=VERTICES, faces=faces, verticesPerFace=3)
shape_obj.specular = 0.0
shape_obj.diffuse = 0.9

figure = vv.gcf()

axes = vv.gca()
axes.bgcolor = (0, 0, 0)
axes.axis.showGrid = False
axes.axis.visible = False

axes.camera = '3d'
axes.camera.fov = 60
axes.camera.zoom = 0.1

axes.light0.Off()

# Create a fixed light source
light_obj = axes.lights[1]
light_obj.On()
light_obj.position = (5.0, 5.0, 5.0, 0.0)

comet_images = []

for azm_angle in tqdm(range(300)):

    axes.camera.azimuth = 360 * float(azm_angle) / 300

    axes.Draw()
    figure.DrawNow()

    temp_image = vv.getframe(vv.gca())
    comet_images.append((temp_image*255).astype(np.uint8))

imageio.mimsave('comet_67P.gif', comet_images, duration=0.04)